In [3]:
import mlx.core as mx
from torch_geometric.utils.sparse import index2ptr
from torch_geometric.datasets import Planetoid
from scipy.sparse import csr_matrix
from torch_geometric.nn import Node2Vec
import torch
from torch_geometric.utils import is_undirected

Initialize Cora dataset

In [4]:
dataset = Planetoid(root ="data/Cora", name='Cora')

Processing...
Done!


convert the coo matrix into csr

In [5]:
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_geometric.utils import sort_edge_index
from torch_geometric.utils.sparse import index2ptr
from torch.utils.data import DataLoader

In [6]:
data, edge_index = dataset.x, dataset.edge_index

In [7]:
!pip install torch_cluster

In [8]:
from torch_cluster.rw import random_walk

In [9]:
num_nodes = maybe_num_nodes(edge_index=edge_index)
loader = DataLoader(range(num_nodes), batch_size=1000)

In [10]:
start = next(iter(loader))

In [11]:
import time

####  Benchmarking torch cluster for generating 1000 random walks ####

In [12]:
start_time = time.time()
num_nodes  = maybe_num_nodes(edge_index=edge_index)
row, col = sort_edge_index(edge_index=edge_index, num_nodes=num_nodes)
row_ptr, col = index2ptr(row, num_nodes), col
print(type(col), type(row_ptr))
random_walks = torch.ops.torch_cluster.random_walk(row_ptr, col, start, 1000, 1.0, 1.0)
print("Time taken to perform 1000 random walks with Torch_cluster is", time.time()-start_time)

<class 'torch.Tensor'> <class 'torch.Tensor'>
Time taken to perform 1000 random walks with Torch_cluster is 0.04140591621398926


Even with CPU, torch cluster's effecient implementation can generate 1000 random walks pretty quickly. The reason behind `torch_cluster` being so effecient lies in its optimized parallel processing algorithms written in c++
~~~cpp
auto rand = torch::rand({numel, walk_length});
  auto rand_data = rand.data_ptr<float>();

  int64_t grain_size = at::internal::GRAIN_SIZE / walk_length;
  at::parallel_for(0, numel, grain_size, [&](int64_t begin, int64_t end) {
    for (auto n = begin; n < end; n++) {
      int64_t n_cur = start[n], e_cur, row_start, row_end, idx;

      n_out[n * (walk_length + 1)] = n_cur;

      for (auto l = 0; l < walk_length; l++) {
        row_start = rowptr[n_cur], row_end = rowptr[n_cur + 1];
        if (row_end - row_start == 0) {
          e_cur = -1;
        } else {
          idx = int64_t(rand_data[n * walk_length + l] * (row_end - row_start));
          e_cur = row_start + idx;
          n_cur = col[e_cur];
        }
        n_out[n * (walk_length + 1) + (l + 1)] = n_cur;
        e_out[n * walk_length + l] = e_cur;
      }
    }
  });
~~~ 

The above code generates the threads which are defined in `ATen` library by pytorch for its C++ api, which then iterate through each of the nodes neighbors and generate a random walk.

#### Benchmarking random walks using numpy ####

In [13]:
import numpy as np

Create own random walk algorithm and measure the time taken on CPU

In [14]:
def random_walk_numpy_optimized(rowptr, col, start, walk_length, rand_data):
    num_walks = len(start)
    num_nodes = len(rowptr) - 1
    
    n_out = np.zeros((num_walks, walk_length + 1), dtype=np.int64) 
    e_out = np.zeros((num_walks, walk_length), dtype=np.int64)
    
    n_out[:, 0] = start
    
    for l in range(walk_length):
        n_cur = n_out[:, l]
        row_start = rowptr[n_cur]
        row_end = rowptr[n_cur + 1]
        
        mask = (row_end - row_start) > 0
        num_neighbors = row_end - row_start
        
        rand_idx = (rand_data[l::walk_length] * num_neighbors).astype(np.int64)
        e_cur = row_start + rand_idx
        n_cur = col[e_cur]
        
        n_cur[~mask] = n_out[~mask, l]
        e_cur[~mask] = -1
        
        n_out[:, l + 1] = n_cur
        e_out[:, l] = e_cur
    
    return n_out, e_out

In [15]:
start_time = time.time()
num_nodes = maybe_num_nodes(edge_index=edge_index)
row, col = sort_edge_index(edge_index=edge_index, num_nodes=num_nodes)
row_numpy = row.numpy()
unique_vals, counts = np.unique(row_numpy, return_counts=True)
row_ptr_numpy = np.cumsum(counts)
row_ptr_numpy = np.insert(row_ptr_numpy, 0, 0)
rand_data = np.random.rand(1000, 1000).astype(np.float32).flatten()
random_walk_numpy_optimized(row_ptr_numpy, col.numpy(), start.numpy(), walk_length=1000, rand_data=rand_data)
print("time taken for random walks using numpy is ", time.time()-start_time)

time taken for random walks using numpy is  0.07817220687866211


Numpy's effeciency is quite slow as compared to that of torc_cluster by 100x. Although the speed can be improved by using a JIT compiler like Numba

### Benchmarking simulations for mlx ###

Can we achieve similar results in MLX?. Although torch_cluster seems to utilize low level parallel optimizations to speed up computation. It still relies on CPU and nvidia GPU for faster processing. Since MLX can interface with GPU's directly, we should be able to speed up the process and achieve comparable performance with mlx

In [16]:
from mlx_graphs.datasets import PlanetoidDataset
from mlx_graphs.utils.sorting import sort_edge_index

In [17]:
cora_dataset = PlanetoidDataset(name='cora', base_dir="~")

Loading cora data ... Done


In [18]:
edge_index = cora_dataset.graphs[0].edge_index

In [19]:
@mx.compile
def random_walk_mlx(row_ptr:mx.array, col: mx.array, start:mx.array, walk_length: int, rand_data):
    num_walks = len(start)
    num_nodes = row_ptr.shape[0]-1
    n_out = mx.zeros((num_walks, walk_length + 1), dtype=col.dtype)
    e_out = mx.zeros((num_walks, walk_length), dtype=col.dtype)

    n_out[:, 0] = start

    for l in range(walk_length):
        n_cur = n_out[:, l]

        row_start = row_ptr[n_cur]
        row_end = row_ptr[n_cur + 1]

        mask = (row_end - row_start) > 0
        num_neighbors = row_end - row_start

        rand_idx = (rand_data[l::walk_length] * num_neighbors)
        rand_idx = rand_idx.astype(mx.int64)
        e_cur = row_start + rand_idx
        n_cur = col[e_cur]
        n_cur = mx.where(~mask, n_out[:, l], n_cur)
        e_cur = mx.where(~mask, -1, e_cur)

        n_out[:, l + 1] = n_cur
        e_out[:, l] = e_cur

    return n_out, e_out

In [24]:
start_time  = time.time()
num_nodes = cora_dataset.graphs[0].num_nodes
sorted_edge_index = sort_edge_index(edge_index=edge_index)
row_mlx = sorted_edge_index[0][0]
col_mlx = sorted_edge_index[0][1]
unique_vals, counts_mlx = np.unique(np.array(row_mlx, copy = False), return_counts=True)
cum_sum_mlx = counts_mlx.cumsum()
row_ptr_mlx = mx.concatenate([mx.array([0]),mx.array(cum_sum_mlx)])
rand_data = mx.random.uniform(shape = [1000, 1000]).flatten()
random_walk_mlx(row_ptr_mlx, col_mlx, start=mx.array(start.numpy()), walk_length=1000, rand_data=rand_data)
print("Time taken by mlx and numpy is ", time.time()-start_time)

Time taken by mlx and numpy is  0.03845572471618652


Using `mx.compile`, we can speed up the computation y almost 3x. ALthough first iteration of compilation will be slow as mlx will build the compiled graph, optimize it and then generate compiled code. After the graph is build, computation is significantly faster